# Programación Dinámica

La idea es dividir el problema en subproblemas de tamaños menores que se resuelven recursivamente (o iterativamente). Al igual que divide and conquer.

En BT nos encontramos que se realizan muchas veces llamadas a la función recursiva con los mismo parámetros, esto se conoce como SUPERPOSICIÓN DE ESTADOS, donde el arbol de llamadas recursivas resuelve el mismo problema varias veces.
Un algoritmo de PD evita estas repeticiones con alguno de estos dos esquemas:
- Top-down: Se implementa recursivamente, guardadno el resultado de cada llamada recursiva en una estructura de datos (memoización). Si una llamada recurisva se repite, se toma el resultado de esta estructura.
- Bottom-up: Resolvemos primero los subproblemas mas pequeños y guardamos todos los resultados.

### Calculo de coeficientes binomiales

Usando el teorema de la teórica, podemos implemetar

In [2]:
import numpy as np
import time

def combBT(n, k):
    if k == 0 or k == n:
        return 1
    else:
        return combBT(n-1, k-1) + combBT(n-1, k)

def comb(n, k):
    A = np.zeros((n+1, k+1))
    for i in range(1, n+1):
        A[i][0] = 1
    for j in range(0, k+1):
        A[j][j] = 1
    for i in range(2, n+1):
        for j in range(1, min(i, k+1)):
            A[i][j] = A[i-1][j-1] + A[i-1][j]
    return A[n][k]

start_time_bt = time.time()
result_bt = combBT(30, 15)
end_time_bt = time.time()
print(f"Resultado combBT: {result_bt}, Tiempo: {end_time_bt - start_time_bt} segundos")

start_time_comb = time.time()
result_comb = comb(30, 15)
end_time_comb = time.time()
print(f"Resultado comb: {result_comb}, Tiempo: {end_time_comb - start_time_comb} segundos")

Resultado combBT: 155117520, Tiempo: 18.90801239013672 segundos
Resultado comb: 155117520.0, Tiempo: 0.0 segundos


Donde la tecnica usada es Bottom-up, ya que empezamos desde "abajo" o bien partimos de los subproblemas al problema (empezamos desde n=0 y k=0 hasta n=n y k=n).
Logramos reducir la complejidad del algoritmo desde $Ω=(\binom{n}{k})$ a $O(nk)$ con PD ya que recorremos y calculamos solo valores para la matriz de $n*k$. Su complejidad espacial es $O(nk)$, salvo que usemos solo 2 listas y vayamos guardando y sobreescribiendolas de tal manera de conseguir $O(k)$. Esto se puede ya que utilizamos valores de la fila anterior y por unica vez.

### Problema del cambio

In [30]:
a = [25, 10, 5, 1]

def cambioBT(s):
    if s == 0:
        return 0
    temp = [float('inf')]
    for i in range(len(a)):
        if a[i] <= s:
            temp.append(1 + cambioBT(s-a[i]))
    return min(temp)

def cambioPD(s):
    n = len(a)
    dp = [[float('inf')] * (s + 1) for _ in range(n + 1)]

    # Base: 0 monedas para hacer cambio de 0 centavos
    for i in range(n + 1):
        dp[i][0] = 0

    for i in range(1, n + 1):
        for j in range(1, s + 1):
            if a[i - 1] <= j:
                dp[i][j] = min(dp[i-1][j], dp[i][j - a[i - 1]] + 1)
            else:
                dp[i][j] = dp[i-1][j]

    return dp[n][s]

def cambioPD2(s):
    # Crear una tabla para almacenar el número mínimo de monedas para cada valor de 0 a s
    dp = [float('inf')] * (s + 1)
    dp[0] = 0  # Base: 0 monedas para hacer cambio de 0 centavos

    for i in range(1, s + 1):
        for coin in a:
            if coin <= i:
                dp[i] = min(dp[i], dp[i - coin] + 1)

    return dp[s]
    
start_time_bt = time.time()
result_bt = cambioBT(49)
end_time_bt = time.time()
print(f"Resultado cambioBT: {result_bt}, Tiempo: {end_time_bt - start_time_bt} segundos")

start_time_comb = time.time()
result_pd = cambioPD(12431)
end_time_comb = time.time()
print(f"Resultado cambioPD: {result_pd}, Tiempo: {end_time_comb - start_time_comb} segundos")
start_time_comb = time.time()
result_pd2 = cambioPD2(12431)
end_time_comb = time.time()
print(f"Resultado cambioPD2: {result_pd2}, Tiempo: {end_time_comb - start_time_comb} segundos")

Resultado cambioBT: 7, Tiempo: 2.8788881301879883 segundos
Resultado cambioPD: 499, Tiempo: 0.01957225799560547 segundos
Resultado cambioPD2: 499, Tiempo: 0.010388374328613281 segundos


En cambioPD2 tenemos una lista de 0 hasta s, inicializados en infinito. Aca definimos cuantas monedas necesitamos para cada numero de centavos s como indice en la lista. Por cada valor de s, recorremos todas las monedas y vemos cual minimiza mas, usando el valor del minimo de monedas para los valores anteriores de s.

cambioPD y cambioPD2 son ambos Bottom-up, sin embargo tenemos una matriz y una lista como estructuras de datos, para la memoización, respectivamente. Ambos tienen una complejidad de $O(n*s)$, pero con complejidad espacial $O(n*s)$ (por ser matriz de n*s) y $O(s)$ (por ser una lista de s elementos) resp.

Esta complejidad es mucho mejor que la complejidad temporal $O(|a|^s)$ que tenemos en cambioBT. La complejidad surge de que el arbol de recursion se divide en |a| ramas y en el peor caso llega hasta el nivel s.

### Problema de la mochila

Como queremos maximizar, nuestra matriz de memoización debería inicializarse en -infinito. Luego inicializamos la columa 0 y fila 0 en 0 ya que con 0 objetos el maximo beneficio que conseguimos es 0 y con capacidad 0 lo mismo. Ademas necesitamos iniciar desde algun punto.

In [13]:
pesos = [3, 8, 3, 1, 6]
beneficios = [10, 25, 7, 13, 8]
n = 5
C = 10

def m(k, D):
    M = [[float('-inf') for _ in range(D+1)] for _ in range(k+1)]
    
    # Inicializar las columnas y filas base
    for i in range(k+1):
        M[i][0] = 0
    for j in range(D+1):
        M[0][j] = 0
    
    # Rellenar la matriz
    for i in range(1, k+1):
        for j in range(1, D+1):
            if pesos[i-1] <= j:
                M[i][j] = max(M[i-1][j], M[i-1][j-pesos[i-1]] + beneficios[i-1])
            else:
                M[i][j] = M[i-1][j]

    # Reconstrucción de la solución óptima
    solucion = []
    for i in range(n, 0, -1):
        if M[i][C] != M[i-1][C]:
            solucion.append(i-1)
            C -= pesos[i-1]
    
    return M[k][D], solucion

print(m(n, C))


(38, [3, 1])


Finalmente, calculamos todos los valores de la matriz de $(n+1)*(C+1)$ en $O(1)$. Luego tenemos una complejidad temporal y espacial de $O(nC)$.

Notar que M[k][D] proporciona el valor optimo, pero no la solucion optima. Para esto deberiamos recontruir la solución, donde obtendremos el conjunto de objetos que resulta en el valor optimo. Tendremos que recorrer al revez (de n a 0) y partir de j=C, ya que empezamos buscando obtener el beneficio maximo con la capacidad maxima, ir encontrando diferencias entre i y i-1 (agregar objetos obtuvo mas beneficio) y reducir la capacidad con el peso de aquel objeto que maximiza.

### Subsecuencia común más larga



In [20]:
def scml(A, B):
    r = len(A)
    s = len(B)
    l = [[float('-inf') for _ in range(s+1)] for _ in range(r+1)]
    
    for i in range(r+1):
        l[i][0] = 0
    for j in range(s+1):
        l[0][j] = 0
    
    for i in range(1, r+1):
        for j in range(1, s+1):
            if A[i-1] == B[j-1]:
                l[i][j] = l[i-1][j-1] + 1
            else:
                l[i][j] = max(l[i-1][j], l[i][j-1])

    # Reconstruir la solución óptima
    scml_solucion = []
    i, j = r, s
    while i > 0 and j > 0:
        if A[i-1] == B[j-1]:
            scml_solucion.append(A[i-1])
            i -= 1
            j -= 1
        elif l[i-1][j] >= l[i][j-1]:
            i -= 1
        else:
            j -= 1
    # La solución se reconstruyó de atrás hacia adelante, por lo que hay que invertirla
    scml_solucion.reverse()  

    return l[r][s], scml_solucion

A = [9, 5, 2, 8, 7, 3, 1, 6, 4]
B = [2, 9, 3, 5, 8, 7, 4, 1, 6]

longitud, subsecuencia = scml(A, B)
print("Longitud de la SCML:", longitud)
print("Subsecuencia común más larga:", subsecuencia)

Longitud de la SCML: 6
Subsecuencia común más larga: [9, 5, 8, 7, 1, 6]
